In [ ]:
import os
import sys
# Beispiel: setze Projekt-Root manuell (Pfad anpassen)
project_root = os.path.abspath('..')  # oder '..' oder '../..' je nach Notebook-Ordner

import json
import joblib
import matplotlib.pyplot as plt
import numpy as np
from numpy import random
import pandas as pd
from IPython import display  # WICHTIG für Jupyter Live-Update
from pandasgui import show
from tqdm import tqdm
import seaborn as sns
from pprint import pprint
import pickle
import importlib

import src.execution
import src.model
import src.util

importlib.reload(src.execution)
importlib.reload(src.model)
importlib.reload(src.util)

from src.data import NPY
from tensorflow.keras.models import load_model
from src.execution import CNNValidationExecution, WindowSplittingExecution, CNNTuningExecution, TrAdaBoostR2TrainingExecution, CNNTrainingExecution, TrAdaBoostR2ModelSetup_SingleModel_Execution
from src.model import CNN, build_model_for_TrAdaBoostR2, tradaBoostR2_setup
from src.experiment.multiple_cnn_singla_model_training import MultipleCNNTrainingOnExistingData
from src.experiment.multiple_trAdaBoostR2_trianings_specific_models import MultipleTrAdaBoostR2TrainingSpecificModleOutput
from src.util import timestamp
import importlib

### TradaboostR2 Modells ###

TrAdaBoostR2 Modell mit drei Ausgängen auf den einzelnen Objekten aufbauen.

In [ ]:
dateipfad_femDaten="build\\window_split\\sim_data_preprocessed_offset_multiplitiv\\1744056915\\"
dateipfad_realDaten="build\\window_split\\real-data\\1744062620\\"
model_file = "assets\\models\\untrained\\synthetic-model\\syn-model.h5"
best_learning_rate=0.0005026453831067469
save_folder = os.path.join("build", "tradaboost_model", "training_30_TEST_02")
save_filename="training_09"

modell=tradaBoostR2_setup.TrAdaBoostR2ModelSetup(
    model_path=model_file,
    early_stoppping_model=True,
    patience_model=2,
    learningrate_model=best_learning_rate,
    n_estimators_tradaBoostR2=2,
    save_folder=save_folder,
    epochs_model=1,  # Wert bitte setzen
    batch_size_model=32,  # Wert bitte setzen
    early_stoppping_TraDaBoostR2=False,  # Wert bitte setzen
    learningrate_TraDaBoostR2=1  # Wert bitte setzen
)

training=TrAdaBoostR2TrainingExecution(tradaboostr2_model=modell,
                                                dateipfad_source_daten=dateipfad_femDaten,
                                                dateipfad_target_daten=dateipfad_realDaten,
                                                use_scaled_labels=True,
                                                model_file=model_file,
                                                save_folder=save_folder,
                                                save_filenssame="testmodel",
                                                training_dokumentation=True)


### Tuning der drei Modelle ###

- Modell mit drei Ausgängen verwenden bei denen jweils der input gleich ist. 
- Raum der Suche ist bei allen Modellen gleich, deswegen nur eine matadata.json Datei
- Tuning erfolgt auf unscalierten Daten

In [ ]:
SPLITTED_DATA_FOLDER = "build\\window_split\\sim_data_preprocessed\\1743966827\\"
CNNTuningExecution.execute_tuning_three_model(SPLITTED_DATA_FOLDER,train_on_scled_labels=True)

---
### Basline Modelle trainieren ###

- Modell getuned auf den SImulationsdaten
- Training auf den Realdaten und Auswertung auf den Realdaten
- Split 80/20
- 15 Datensätze verwenden, mit k-Fold

Daten erzeugen

In [ ]:
def validation_data_Mix(N: int, anteil):
    for seed in random.randint(0, 32000, N):
        DATA: str = "assets/data/real-data.csv"
        VALIDATION_SPLIT: bool = True # If the data should also be splitted into a validation set?
        TEST_SIZE: float = (100-anteil)/100
        BATCH_SPLIT: bool = True
        BATCHSIZE: int = 1800 # real-data: 1800, synthetic-data: 326
        SEED: int = seed # Seed for random state -> Split with same seed and data will always result in the same split
        INTERPOLATION: bool = False
        WINDOWSIZE: int = 10
        SEP: str = ";" # Separator of the csv file
        DECIMAL: str = "," # Decimal separator of the csv file
        
        WindowSplittingExecution.execute_with_scaler(DATA, 
                                        BATCH_SPLIT, 
                                        VALIDATION_SPLIT, 
                                        TEST_SIZE, 
                                        SEED, 
                                        BATCHSIZE, 
                                        INTERPOLATION, 
                                        WINDOWSIZE, 
                                        SEP, 
                                        DECIMAL)


In [ ]:
validation_data_Mix(N=15, anteil=80)
# Alle anderen Daten sind bereits erzeugt

## Training auf den Realdaten ##
Training auf den Datenanteilen von 5%, 10%, 15%, 20%, 50%, 80%
Siehe .py in den unterordnern

In [ ]:
#5% Realdaten
dateipfad_realDaten="build\\window_split\\real-data\\Realdaten_5Prozent_Random_N15"
dateipfad_modell="assets\\models\\untrained\\seperated-models"
ergebnis_ordner="build\\train_single_models\\training_1_Realdaten_5Prozent_N15"
MultipleCNNTrainingOnExistingData(target_folder=ergebnis_ordner,folder_data=dateipfad_realDaten, folder_base_model=dateipfad_modell, 
                                  result_folder_name_initial="training_1_SpecificModel")

In [ ]:
Model_Path="assets\\models\\untrained\\seperated-models"
dateipfad_realDaten="build\\window_split\\real-data\\1744062620\\"
CNNTrainingExecution.execute_three_models_training(model_folder=Model_Path, data_file=dateipfad_realDaten)

---
### TrAdaBoostR2 bei spezifischen Modellen für ein Output ###
- Tradaboost trainiert auf skalierten targets, nicht wie die single models, auf unskalierten Targets.
- ACHTUNG: Validierung des MAE auf den NICHT skalierten Daten, also dann in mm bzw rad oder grad, nciht sicher für Phi

In [ ]:
dateipfad_femDaten="build\\window_split\\sim_data_preprocessed_offset_multiplitiv\\1744056915\\"
dateipfad_realDaten="build\\window_split\\real-data\\1744062620\\"
model_file = "assets\\models\\untrained\\seperated-models"
save_folder = os.path.join("build", "tradaboost_model", "training_30_TEST_02")
save_filename="training_09"

modell=tradaBoostR2_setup.TrAdaBoostR2ModelSetup(
    model_path=model_file,
    early_stoppping_model=True,
    patience_model=2,
    learningrate_model=None,
    n_estimators_tradaBoostR2=2,
    save_folder=save_folder,
    epochs_model=1,  # Wert bitte setzen
    batch_size_model=32,  # Wert bitte setzen
    early_stoppping_TraDaBoostR2=False,  # Wert bitte setzen
    learningrate_TraDaBoostR2=1  # Wert bitte setzen
)

training=TrAdaBoostR2ModelSetup_SingleModel_Execution(tradaboostr2_model=modell,
                                                dateipfad_source_daten=dateipfad_femDaten,
                                                dateipfad_target_daten=dateipfad_realDaten,
                                                use_scaled_labels=True,
                                                model_file=model_file,
                                                save_folder=save_folder,
                                                save_filename="testmodel",
                                                training_dokumentation=False)


Debuggen TradaBoostR2Training

In [ ]:
dateipfad_femDaten="build\\window_split\\sim_data_preprocessed_offset_multiplitiv\\1744056915\\"
dateipfad_realDaten="build\\window_split\\real-data\\Realdaten_5Prozent_Random_N15\\"
model_file = "assets\\models\\untrained\\seperated-models"
save_folder = os.path.join("build", "tradaboost_model_specifiv_for_output", "trAdaBoostR2_1_Realdaten_5Prozent_N15")

MultipleTrAdaBoostR2TrainingSpecificModleOutput(target_data_folder=dateipfad_realDaten, 
                                                source_data_folder=dateipfad_femDaten,
                                                folder_base_model=model_file,
                                                result_folder=save_folder,
                                                result_folder_name_initial="trAdaBoostR2_1_SpecificModel")